In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame([], columns=['a', 'b'])

In [3]:
df

,a,b


In [4]:
df.loc[0] = [1, 2]
df.loc[1] = [3, 4]

In [8]:
for x in df.itertuples():
    print(type(x))
    print(x)

<class 'pandas.core.frame.Pandas'>
Pandas(Index=0, a=1, b=2)
<class 'pandas.core.frame.Pandas'>
Pandas(Index=1, a=3, b=4)


In [2]:
import numpy as np

x = ["a", "a", "b", "a", "c"]
print(np.unique(x, return_inverse=True))

(array(['a', 'b', 'c'], dtype='<U1'), array([0, 0, 1, 0, 2], dtype=int64))


In [1]:
import pickle
from hyperopt import Trials

with open("bayes_trials.bin", "rb") as f:
    trials = pickle.load(f)

In [2]:
trials.losses()

[-0.6413659567404406, -0.07993643722492222, -0.645220652929391]

In [23]:
import math
import numpy as np
print(math.log10(math.exp(-5)))
print(np.log(1e-5))

-2.171472409516259
-11.512925464970229


NameError: name 'np' is not defined